In [1]:
#check CUDA und d3rlpy Installation
!nvidia-smi
!nvcc --version
import torch 
print(torch.version.cuda)  # sollte '12.2' oder ähnlich anzeigen
print(torch.cuda.is_available())  # sollte True sein
print(torch.cuda.current_device())  # sollte 0 sein, wenn eine GPU verfügbar ist
import d3rlpy
print(d3rlpy.__version__)

Mon Nov 17 21:18:57 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 539.56                 Driver Version: 539.56       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P4                     TCC   | 00000000:06:10.0 Off |                    0 |
| N/A   42C    P0              23W /  75W |    638MiB /  7680MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\NilsWindows\Desktop\research2.0\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.8.1


In [2]:
import d3rlpy
import numpy as np


# vector observation
# 1000 steps of observations with shape of (100,)
observations = np.random.random((1000, 100))

# 1000 steps of actions with shape of (4,)
actions = np.random.random((1000, 4))

# 1000 steps of rewards
rewards = np.random.random(1000)

# 1000 steps of terminal flags
terminals = np.random.randint(2, size=1000)

dataset = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminals,
)

dataset1, env = d3rlpy.datasets.get_dataset("hopper-medium-v0")


# if you don't use GPU, set device=None instead.
sac = d3rlpy.algos.SACConfig().create(device='cuda:0')

# initialize neural networks with the given observation shape and action size.
# this is not necessary when you directly call fit or fit_online method.
sac.build_with_dataset(dataset)
# calculate metrics with training dataset
td_error_evaluator = d3rlpy.metrics.TDErrorEvaluator(episodes=dataset.episodes)

# evaluate algorithm on the environment
#rewards = env_evaluator(sac, dataset=None)

sac.fit(
    dataset,
    n_steps=1000,
    evaluators={
        'td_error': td_error_evaluator,
    }
)

# get first observation from the dataset
observation = observations[0]

# return actions based on the greedy-policy
action = sac.predict(np.expand_dims(observation, axis=0))

# estimate action-values
value = sac.predict_value(np.expand_dims(observation, axis=0), action)

# save full parameters and configurations in a single file.
sac.save('sac.d3')
# load full parameters and build algorithm
sac2 = d3rlpy.load_learnable("sac.d3")

# save full parameters only
sac.save_model('sac.pt')

# save the greedy-policy as TorchScript
sac.save_policy('policy.pt')

2025-11-17 21:19.11 [info     ] Signatures have been automatically determined. action_signature=Signature(dtype=[dtype('float64')], shape=[(4,)]) observation_signature=Signature(dtype=[dtype('float64')], shape=[(100,)]) reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)])
2025-11-17 21:19.11 [info     ] Action-space has been automatically determined. action_space=<ActionSpace.CONTINUOUS: 1>
2025-11-17 21:19.11 [info     ] Action size has been automatically determined. action_size=4



You appear to be missing MuJoCo.  We expected to find the file here: C:\Users\NilsWindows\.mujoco\mujoco210

This package only provides python bindings, the library must be installed separately.

Please follow the instructions on the README to install MuJoCo

    https://github.com/openai/mujoco-py#install-mujoco

Which can be downloaded from the website

    https://www.roboti.us/index.html



Exception: 
You appear to be missing MuJoCo.  We expected to find the file here: C:\Users\NilsWindows\.mujoco\mujoco210

This package only provides python bindings, the library must be installed separately.

Please follow the instructions on the README to install MuJoCo

    https://github.com/openai/mujoco-py#install-mujoco

Which can be downloaded from the website

    https://www.roboti.us/index.html
